In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report 
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC


In [2]:
data = pd.read_csv('final_with_covid_AS.csv')
data.head()

,CountCases,BedDys,BeddystoCC,Severity,DiedCases,DiedBdDys,Hsptlsns,April,August,December,...,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021,CovidPrd
0,17,133,7.823529,75,4,9,13,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,13,372,28.615385,63,4,34,9,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,106,13.250000,48,4,13,4,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,45,218,4.844444,153,3,24,42,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,24,238,9.916667,90,3,8,21,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [3]:
filtered_data2 = data[data['CarPngr'] == 1]
filtered_data2.info()


<class 'pandas.core.frame.DataFrame'>
Index: 11789 entries, 33 to 54030
Data columns (total 58 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CountCases       11789 non-null  int64  
 1   BedDys           11789 non-null  int64  
 2   BeddystoCC       11789 non-null  float64
 3   Severity         11789 non-null  int64  
 4   DiedCases        11789 non-null  int64  
 5   DiedBdDys        11789 non-null  int64  
 6   Hsptlsns         11789 non-null  int64  
 7   April            11789 non-null  int64  
 8   August           11789 non-null  int64  
 9   December         11789 non-null  int64  
 10  February         11789 non-null  int64  
 11  January          11789 non-null  int64  
 12  July             11789 non-null  int64  
 13  June             11789 non-null  int64  
 14  March            11789 non-null  int64  
 15  May              11789 non-null  int64  
 16  November         11789 non-null  int64  
 17  October         

In [4]:
num_bins = 2

bed_days_min = filtered_data2['Severity'].min()
bed_days_max = filtered_data2['Severity'].max()

bin_width = (bed_days_max - bed_days_min) / num_bins
bins = [bed_days_min + i * bin_width for i in range(num_bins + 1)]

def bin_function(x):
    for i in range(1, len(bins)):
        if x <= bins[i]:
            return i - 1
    return i

filtered_data2['Bed_days_category'] = filtered_data2['Severity'].apply(bin_function)

/var/folders/3l/xh28kn0j7_7d5hh7fgz188940000gn/T/ipykernel_1430/530833198.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data2['Bed_days_category'] = filtered_data2['Severity'].apply(bin_function)


In [5]:

from xgboost import XGBClassifier

X = filtered_data2[['Age1725','Age2639','Age4064','Age6574','Age75plus',
                   
                   'Male','Female',
                   
                   'Regional','MajorCities',

                   'April','December','February','January','July',
                   'June','March','May','November','October','September',

                   'Cntrprt23WldMV','CntrprtFxdStnry','CntrprtHvyVhcl',
                   'CntrprtNClsn','CntrprtOther','CntrprtNMV','CntrprtCrTrkVn',
                   'CntrprtPdlC','CntrprtPdstAnml','CntrprtTrn',

                   'Y2014','Y2015','Y2016','Y2017','Y2018','Y2019','Y2020','Y2021',
                   'Y2013','Y2012',
                   'CovidPrd']]



y = filtered_data2['Bed_days_category']


In [6]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE, ADASYN  # Import SMOTE and ADASYN
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_smote = []
precision_list_smote = []
recall_list_smote = []
f1_list_smote = []
entropy_list_smote = []
confusion_matrices_smote = []

accuracy_list_adasyn = []
precision_list_adasyn = []
recall_list_adasyn = []
f1_list_adasyn = []
entropy_list_adasyn = []
confusion_matrices_adasyn = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the Logistic Regression model without SMOTE/ADASYN
    logreg_model = LogisticRegression(random_state=42, max_iter=1000)
    logreg_model.fit(X_train, y_train)
    
    # Predictions and probabilities before SMOTE/ADASYN
    y_pred_before = logreg_model.predict(X_test)
    y_prob_before = logreg_model.predict_proba(X_test)
    
    # Calculate entropy before SMOTE/ADASYN
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before SMOTE/ADASYN
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Apply SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    
    # Initialize and train the Logistic Regression model with SMOTE
    logreg_model_smote = LogisticRegression(random_state=42, max_iter=1000)
    logreg_model_smote.fit(X_train_smote, y_train_smote)
    
    # Predictions and probabilities after SMOTE
    y_pred_smote = logreg_model_smote.predict(X_test)
    y_prob_smote = logreg_model_smote.predict_proba(X_test)
    
    # Calculate entropy after SMOTE
    entropy_smote = calculate_entropy(y_prob_smote)
    entropy_list_smote.append(entropy_smote)
    
    # Store metrics after SMOTE
    accuracy_list_smote.append(accuracy_score(y_test, y_pred_smote))
    precision_list_smote.append(precision_score(y_test, y_pred_smote))
    recall_list_smote.append(recall_score(y_test, y_pred_smote))
    f1_list_smote.append(f1_score(y_test, y_pred_smote))
    confusion_matrices_smote.append(confusion_matrix(y_test, y_pred_smote))
    
    # Apply ADASYN to the training data
    adasyn = ADASYN(random_state=42)
    X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)
    
    # Initialize and train the Logistic Regression model with ADASYN
    logreg_model_adasyn = LogisticRegression(random_state=42, max_iter=1000)
    logreg_model_adasyn.fit(X_train_adasyn, y_train_adasyn)
    
    # Predictions and probabilities after ADASYN
    y_pred_adasyn = logreg_model_adasyn.predict(X_test)
    y_prob_adasyn = logreg_model_adasyn.predict_proba(X_test)
    
    # Calculate entropy after ADASYN
    entropy_adasyn = calculate_entropy(y_prob_adasyn)
    entropy_list_adasyn.append(entropy_adasyn)
    
    # Store metrics after ADASYN
    accuracy_list_adasyn.append(accuracy_score(y_test, y_pred_adasyn))
    precision_list_adasyn.append(precision_score(y_test, y_pred_adasyn))
    recall_list_adasyn.append(recall_score(y_test, y_pred_adasyn))
    f1_list_adasyn.append(f1_score(y_test, y_pred_adasyn))
    confusion_matrices_adasyn.append(confusion_matrix(y_test, y_pred_adasyn))
    
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Oversampling:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_smote)} After SMOTE:\n', confusion_matrices_smote[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_adasyn)} After ADASYN:\n', confusion_matrices_adasyn[-1])

# Calculate mean and standard deviation of each metric before oversampling
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after SMOTE
mean_accuracy_smote = np.mean(accuracy_list_smote)
std_accuracy_smote = np.std(accuracy_list_smote)
mean_precision_smote = np.mean(precision_list_smote)
std_precision_smote = np.std(precision_list_smote)
mean_recall_smote = np.mean(recall_list_smote)
std_recall_smote = np.std(recall_list_smote)
mean_f1_smote = np.mean(f1_list_smote)
std_f1_smote = np.std(f1_list_smote)
mean_entropy_smote = np.mean(entropy_list_smote)
std_entropy_smote = np.std(entropy_list_smote)
# Calculate mean and standard deviation of each metric after ADASYN
mean_accuracy_adasyn = np.mean(accuracy_list_adasyn)
std_accuracy_adasyn = np.std(accuracy_list_adasyn)
mean_precision_adasyn = np.mean(precision_list_adasyn)
std_precision_adasyn = np.std(precision_list_adasyn)
mean_recall_adasyn = np.mean(recall_list_adasyn)
std_recall_adasyn = np.std(recall_list_adasyn)
mean_f1_adasyn = np.mean(f1_list_adasyn)
std_f1_adasyn = np.std(f1_list_adasyn)
mean_entropy_adasyn = np.mean(entropy_list_adasyn)
std_entropy_adasyn = np.std(entropy_list_adasyn)

# Calculate mean confusion matrix before and after SMOTE/ADASYN
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_smote = np.mean(confusion_matrices_smote, axis=0)
mean_conf_matrix_adasyn = np.mean(confusion_matrices_adasyn, axis=0)

# Print results before oversampling
print('--- Before Oversampling ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after SMOTE
print('--- After SMOTE ---')
print('Mean Accuracy:', mean_accuracy_smote)
print('Accuracy Std Dev:', std_accuracy_smote)
print('Mean Precision:', mean_precision_smote)
print('Precision Std Dev:', std_precision_smote)
print('Mean Recall:', mean_recall_smote)
print('Recall Std Dev:', std_recall_smote)
print('Mean F1-score:', mean_f1_smote)
print('F1-score Std Dev:', std_f1_smote)
print('Mean Entropy:', mean_entropy_smote)
print('Entropy Std Dev:', std_entropy_smote)
print('Mean Confusion Matrix:\n', mean_conf_matrix_smote)

# Print results after ADASYN
print('--- After ADASYN ---')
print('Mean Accuracy:', mean_accuracy_adasyn)
print('Accuracy Std Dev:', std_accuracy_adasyn)
print('Mean Precision:', mean_precision_adasyn)
print('Precision Std Dev:', std_precision_adasyn)
print('Mean Recall:', mean_recall_adasyn)
print('Recall Std Dev:', std_recall_adasyn)
print('Mean F1-score:', mean_f1_adasyn)
print('F1-score Std Dev:', std_f1_adasyn)
print('Mean Entropy:', mean_entropy_adasyn)
print('Entropy Std Dev:', std_entropy_adasyn)
print('Mean Confusion Matrix:\n', mean_conf_matrix_adasyn)

# Calculate mean and standard deviation of each metric after ADASYN


Confusion Matrix for Fold 1 Before Oversampling:
 [[1156    3]
 [  10   10]]
Confusion Matrix for Fold 1 After SMOTE:
 [[1119   40]
 [   1   19]]
Confusion Matrix for Fold 1 After ADASYN:
 [[1120   39]
 [   2   18]]
Confusion Matrix for Fold 2 Before Oversampling:
 [[1156    3]
 [   6   14]]
Confusion Matrix for Fold 2 After SMOTE:
 [[1121   38]
 [   2   18]]
Confusion Matrix for Fold 2 After ADASYN:
 [[1120   39]
 [   2   18]]
Confusion Matrix for Fold 3 Before Oversampling:
 [[1154    5]
 [  11    9]]
Confusion Matrix for Fold 3 After SMOTE:
 [[1104   55]
 [   3   17]]
Confusion Matrix for Fold 3 After ADASYN:
 [[1102   57]
 [   3   17]]
Confusion Matrix for Fold 4 Before Oversampling:
 [[1155    4]
 [   5   15]]
Confusion Matrix for Fold 4 After SMOTE:
 [[1119   40]
 [   2   18]]
Confusion Matrix for Fold 4 After ADASYN:
 [[1119   40]
 [   2   18]]
Confusion Matrix for Fold 5 Before Oversampling:
 [[1155    4]
 [   8   12]]
Confusion Matrix for Fold 5 After SMOTE:
 [[1104   55]
 [  

In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
import numpy as np

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
uncertainty_list = []  # List to store entropy for each fold
confusion_matrices = []

# Choose an oversampling method
# oversampler = SMOTE(random_state=42)
# oversampler = ADASYN(random_state=42)
oversampler = RandomOverSampler(random_state=42)

# Function to calculate entropy
def calculate_entropy(probabilities):
    entropy = -np.sum(probabilities * np.log(probabilities + 1e-9), axis=1)  # Add small value to avoid log(0)
    return np.mean(entropy)

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Apply oversampling
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
    
    # Initialize and train the Logistic Regression model
    logreg_model = LogisticRegression(random_state=42, max_iter=1000)
    logreg_model.fit(X_train_resampled, y_train_resampled)
    
    # Predict probabilities and classes
    y_pred_logreg = logreg_model.predict(X_test)
    y_prob_logreg = logreg_model.predict_proba(X_test)
    
    # Calculate and store entropy
    entropy = calculate_entropy(y_prob_logreg)
    uncertainty_list.append(entropy)
    
    # Store metrics
    accuracy_list.append(accuracy_score(y_test, y_pred_logreg))
    precision_list.append(precision_score(y_test, y_pred_logreg))
    recall_list.append(recall_score(y_test, y_pred_logreg))
    f1_list.append(f1_score(y_test, y_pred_logreg))
    
    # Store confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred_logreg)
    confusion_matrices.append(conf_matrix)
    print(f'Confusion Matrix for Fold {len(confusion_matrices)}:\n', conf_matrix)

# Calculate mean and standard deviation of each metric
mean_accuracy = np.mean(accuracy_list)
std_accuracy = np.std(accuracy_list)
mean_precision = np.mean(precision_list)
std_precision = np.std(precision_list)
mean_recall = np.mean(recall_list)
std_recall = np.std(recall_list)
mean_f1 = np.mean(f1_list)
std_f1 = np.std(f1_list)

# Calculate mean and standard deviation of entropy
mean_entropy = np.mean(uncertainty_list)
std_entropy = np.std(uncertainty_list)

# Calculate mean confusion matrix
mean_conf_matrix = np.mean(confusion_matrices, axis=0)

# Print results
print('Mean Accuracy:', mean_accuracy)
print('Accuracy Std Dev:', std_accuracy)
print('Mean Precision:', mean_precision)
print('Precision Std Dev:', std_precision)
print('Mean Recall:', mean_recall)
print('Recall Std Dev:', std_recall)
print('Mean F1-score:', mean_f1)
print('F1-score Std Dev:', std_f1)
print('Mean Entropy:', mean_entropy)
print('Entropy Std Dev:', std_entropy)
print('Mean Confusion Matrix:\n', mean_conf_matrix)


Confusion Matrix for Fold 1:
 [[1126   33]
 [   1   19]]
Confusion Matrix for Fold 2:
 [[1114   45]
 [   1   19]]
Confusion Matrix for Fold 3:
 [[1113   46]
 [   0   20]]
Confusion Matrix for Fold 4:
 [[1118   41]
 [   0   20]]
Confusion Matrix for Fold 5:
 [[1119   40]
 [   0   20]]
Confusion Matrix for Fold 6:
 [[1128   31]
 [   1   19]]
Confusion Matrix for Fold 7:
 [[1120   39]
 [   0   20]]
Confusion Matrix for Fold 8:
 [[1104   55]
 [   0   20]]
Confusion Matrix for Fold 9:
 [[1112   47]
 [   1   19]]
Confusion Matrix for Fold 10:
 [[1124   34]
 [   0   20]]
Mean Accuracy: 0.9647982304937424
Accuracy Std Dev: 0.00585221946880008
Mean Precision: 0.3270390979970551
Precision Std Dev: 0.036056142760751184
Mean Recall: 0.9799999999999999
Recall Std Dev: 0.024494897427831803
Mean F1-score: 0.4891742063739285
F1-score Std Dev: 0.040317984563896944
Mean Entropy: 0.057861494381778567
Entropy Std Dev: 0.005608315779413338
Mean Confusion Matrix:
 [[1.1178e+03 4.1100e+01]
 [4.0000e-01 1.960

In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE, ADASYN
import numpy as np
import xgboost as xgb  # Import XGBoost

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_smote = []
precision_list_smote = []
recall_list_smote = []
f1_list_smote = []
entropy_list_smote = []
confusion_matrices_smote = []

accuracy_list_adasyn = []
precision_list_adasyn = []
recall_list_adasyn = []
f1_list_adasyn = []
entropy_list_adasyn = []
confusion_matrices_adasyn = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the XGBoost model without SMOTE/ADASYN
    xgb_model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
    xgb_model.fit(X_train, y_train)
    
    # Predictions and probabilities before SMOTE/ADASYN
    y_pred_before = xgb_model.predict(X_test)
    y_prob_before = xgb_model.predict_proba(X_test)
    
    # Calculate entropy before SMOTE/ADASYN
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before SMOTE/ADASYN
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Apply SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    
    # Initialize and train the XGBoost model with SMOTE
    xgb_model_smote = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
    xgb_model_smote.fit(X_train_smote, y_train_smote)
    
    # Predictions and probabilities after SMOTE
    y_pred_smote = xgb_model_smote.predict(X_test)
    y_prob_smote = xgb_model_smote.predict_proba(X_test)
    
    # Calculate entropy after SMOTE
    entropy_smote = calculate_entropy(y_prob_smote)
    entropy_list_smote.append(entropy_smote)
    
    # Store metrics after SMOTE
    accuracy_list_smote.append(accuracy_score(y_test, y_pred_smote))
    precision_list_smote.append(precision_score(y_test, y_pred_smote))
    recall_list_smote.append(recall_score(y_test, y_pred_smote))
    f1_list_smote.append(f1_score(y_test, y_pred_smote))
    confusion_matrices_smote.append(confusion_matrix(y_test, y_pred_smote))
    
    # Apply ADASYN to the training data
    adasyn = ADASYN(random_state=42)
    X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)
    
    # Initialize and train the XGBoost model with ADASYN
    xgb_model_adasyn = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
    xgb_model_adasyn.fit(X_train_adasyn, y_train_adasyn)
    
    # Predictions and probabilities after ADASYN
    y_pred_adasyn = xgb_model_adasyn.predict(X_test)
    y_prob_adasyn = xgb_model_adasyn.predict_proba(X_test)
    
    # Calculate entropy after ADASYN
    entropy_adasyn = calculate_entropy(y_prob_adasyn)
    entropy_list_adasyn.append(entropy_adasyn)
    
    # Store metrics after ADASYN
    accuracy_list_adasyn.append(accuracy_score(y_test, y_pred_adasyn))
    precision_list_adasyn.append(precision_score(y_test, y_pred_adasyn))
    recall_list_adasyn.append(recall_score(y_test, y_pred_adasyn))
    f1_list_adasyn.append(f1_score(y_test, y_pred_adasyn))
    confusion_matrices_adasyn.append(confusion_matrix(y_test, y_pred_adasyn))
    
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Oversampling:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_smote)} After SMOTE:\n', confusion_matrices_smote[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_adasyn)} After ADASYN:\n', confusion_matrices_adasyn[-1])

# Calculate mean and standard deviation of each metric before oversampling
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after SMOTE
mean_accuracy_smote = np.mean(accuracy_list_smote)
std_accuracy_smote = np.std(accuracy_list_smote)
mean_precision_smote = np.mean(precision_list_smote)
std_precision_smote = np.std(precision_list_smote)
mean_recall_smote = np.mean(recall_list_smote)
std_recall_smote = np.std(recall_list_smote)
mean_f1_smote = np.mean(f1_list_smote)
std_f1_smote = np.std(f1_list_smote)
mean_entropy_smote = np.mean(entropy_list_smote)
std_entropy_smote = np.std(entropy_list_smote)

# Calculate mean and standard deviation of each metric after ADASYN
mean_accuracy_adasyn = np.mean(accuracy_list_adasyn)
std_accuracy_adasyn = np.std(accuracy_list_adasyn)
mean_precision_adasyn = np.mean(precision_list_adasyn)
std_precision_adasyn = np.std(precision_list_adasyn)
mean_recall_adasyn = np.mean(recall_list_adasyn)
std_recall_adasyn = np.std(recall_list_adasyn)
mean_f1_adasyn = np.mean(f1_list_adasyn)
std_f1_adasyn = np.std(f1_list_adasyn)
mean_entropy_adasyn = np.mean(entropy_list_adasyn)
std_entropy_adasyn = np.std(entropy_list_adasyn)

# Calculate mean confusion matrix before and after SMOTE/ADASYN
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_smote = np.mean(confusion_matrices_smote, axis=0)
mean_conf_matrix_adasyn = np.mean(confusion_matrices_adasyn, axis=0)

# Print results before oversampling
print('--- Before Oversampling ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after SMOTE
print('--- After SMOTE ---')
print('Mean Accuracy:', mean_accuracy_smote)
print('Accuracy Std Dev:', std_accuracy_smote)
print('Mean Precision:', mean_precision_smote)
print('Precision Std Dev:', std_precision_smote)
print('Mean Recall:', mean_recall_smote)
print('Recall Std Dev:', std_recall_smote)
print('Mean F1-score:', mean_f1_smote)
print('F1-score Std Dev:', std_f1_smote)
print('Mean Entropy:', mean_entropy_smote)
print('Entropy Std Dev:', std_entropy_smote)
print('Mean Confusion Matrix:\n', mean_conf_matrix_smote)

# Print results after ADASYN
print('--- After ADASYN ---')
print('Mean Accuracy:', mean_accuracy_adasyn)
print('Accuracy Std Dev:', std_accuracy_adasyn)
print('Mean Precision:', mean_precision_adasyn)
print('Precision Std Dev:', std_precision_adasyn)
print('Mean Recall:', mean_recall_adasyn)
print('Recall Std Dev:', std_recall_adasyn)
print('Mean F1-score:', mean_f1_adasyn)
print('F1-score Std Dev:', std_f1_adasyn)
print('Mean Entropy:', mean_entropy_adasyn)
print('Entropy Std Dev:', std_entropy_adasyn)
print('Mean Confusion Matrix:\n', mean_conf_matrix_adasyn)


Confusion Matrix for Fold 1 Before Oversampling:
 [[1154    5]
 [   9   11]]
Confusion Matrix for Fold 1 After SMOTE:
 [[1137   22]
 [   2   18]]
Confusion Matrix for Fold 1 After ADASYN:
 [[1136   23]
 [   2   18]]
Confusion Matrix for Fold 2 Before Oversampling:
 [[1150    9]
 [   5   15]]
Confusion Matrix for Fold 2 After SMOTE:
 [[1133   26]
 [   1   19]]
Confusion Matrix for Fold 2 After ADASYN:
 [[1134   25]
 [   1   19]]
Confusion Matrix for Fold 3 Before Oversampling:
 [[1151    8]
 [  11    9]]
Confusion Matrix for Fold 3 After SMOTE:
 [[1127   32]
 [   8   12]]
Confusion Matrix for Fold 3 After ADASYN:
 [[1125   34]
 [   8   12]]
Confusion Matrix for Fold 4 Before Oversampling:
 [[1154    5]
 [   6   14]]
Confusion Matrix for Fold 4 After SMOTE:
 [[1140   19]
 [   4   16]]
Confusion Matrix for Fold 4 After ADASYN:
 [[1138   21]
 [   4   16]]
Confusion Matrix for Fold 5 Before Oversampling:
 [[1154    5]
 [   8   12]]
Confusion Matrix for Fold 5 After SMOTE:
 [[1125   34]
 [  

In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
from xgboost import XGBClassifier
import numpy as np

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
uncertainty_list = []  # List to store entropy for each fold
confusion_matrices = []

# Choose an oversampling method
# oversampler = SMOTE(random_state=42)
#oversampler = ADASYN(random_state=42)
oversampler = RandomOverSampler(random_state=42)

# Function to calculate entropy
def calculate_entropy(probabilities):
    entropy = -np.sum(probabilities * np.log(probabilities + 1e-9), axis=1)  # Add small value to avoid log(0)
    return np.mean(entropy)

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Apply oversampling
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
    
    xgb_model = XGBClassifier(random_state=42)
    xgb_model.fit(X_train_resampled, y_train_resampled)
    
    # Predict probabilities and classes
    y_pred_xgb = xgb_model.predict(X_test)
    y_prob_xgb = xgb_model.predict_proba(X_test)
    
    # Calculate and store entropy
    entropy = calculate_entropy(y_prob_xgb)
    uncertainty_list.append(entropy)
    
    # Store metrics
    accuracy_list.append(accuracy_score(y_test, y_pred_xgb))
    precision_list.append(precision_score(y_test, y_pred_xgb))
    recall_list.append(recall_score(y_test, y_pred_xgb))
    f1_list.append(f1_score(y_test, y_pred_xgb))
    
    # Store confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred_xgb)
    confusion_matrices.append(conf_matrix)
    print(f'Confusion Matrix for Fold {len(confusion_matrices)}:\n', conf_matrix)

# Calculate mean and standard deviation of each metric
mean_accuracy = np.mean(accuracy_list)
std_accuracy = np.std(accuracy_list)
mean_precision = np.mean(precision_list)
std_precision = np.std(precision_list)
mean_recall = np.mean(recall_list)
std_recall = np.std(recall_list)
mean_f1 = np.mean(f1_list)
std_f1 = np.std(f1_list)

# Calculate mean and standard deviation of entropy
mean_entropy = np.mean(uncertainty_list)
std_entropy = np.std(uncertainty_list)

# Calculate mean confusion matrix
mean_conf_matrix = np.mean(confusion_matrices, axis=0)

# Print results
print('Mean Accuracy:', mean_accuracy)
print('Accuracy Std Dev:', std_accuracy)
print('Mean Precision:', mean_precision)
print('Precision Std Dev:', std_precision)
print('Mean Recall:', mean_recall)
print('Recall Std Dev:', std_recall)
print('Mean F1-score:', mean_f1)
print('F1-score Std Dev:', std_f1)
print('Mean Entropy:', mean_entropy)
print('Entropy Std Dev:', std_entropy)
print('Mean Confusion Matrix:\n', mean_conf_matrix)


Confusion Matrix for Fold 1:
 [[1147   12]
 [   3   17]]
Confusion Matrix for Fold 2:
 [[1145   14]
 [   2   18]]
Confusion Matrix for Fold 3:
 [[1147   12]
 [   9   11]]
Confusion Matrix for Fold 4:
 [[1148   11]
 [   5   15]]
Confusion Matrix for Fold 5:
 [[1150    9]
 [   3   17]]
Confusion Matrix for Fold 6:
 [[1148   11]
 [   6   14]]
Confusion Matrix for Fold 7:
 [[1144   15]
 [   2   18]]
Confusion Matrix for Fold 8:
 [[1144   15]
 [   3   17]]
Confusion Matrix for Fold 9:
 [[1148   11]
 [   7   13]]
Confusion Matrix for Fold 10:
 [[1144   14]
 [   5   15]]
Mean Accuracy: 0.985664450463761
Accuracy Std Dev: 0.0019520587499252233
Mean Precision: 0.5553349588317731
Precision Std Dev: 0.04410526658652206
Mean Recall: 0.775
Recall Std Dev: 0.1101135777277262
Mean F1-score: 0.6447585146086451
F1-score Std Dev: 0.06090575730980049
Mean Entropy: 0.013673904
Entropy Std Dev: 0.0013091504
Mean Confusion Matrix:
 [[1146.5   12.4]
 [   4.5   15.5]]


decesion tree

In [11]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE, ADASYN
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_smote = []
precision_list_smote = []
recall_list_smote = []
f1_list_smote = []
entropy_list_smote = []
confusion_matrices_smote = []

accuracy_list_adasyn = []
precision_list_adasyn = []
recall_list_adasyn = []
f1_list_adasyn = []
entropy_list_adasyn = []
confusion_matrices_adasyn = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the Decision Tree model without SMOTE/ADASYN
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, y_train)
    
    # Predictions and probabilities before SMOTE/ADASYN
    y_pred_before = dt_model.predict(X_test)
    y_prob_before = dt_model.predict_proba(X_test)
    
    # Calculate entropy before SMOTE/ADASYN
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before SMOTE/ADASYN
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Apply SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    
    # Initialize and train the Decision Tree model with SMOTE
    dt_model_smote = DecisionTreeClassifier(random_state=42)
    dt_model_smote.fit(X_train_smote, y_train_smote)
    
    # Predictions and probabilities after SMOTE
    y_pred_smote = dt_model_smote.predict(X_test)
    y_prob_smote = dt_model_smote.predict_proba(X_test)
    
    # Calculate entropy after SMOTE
    entropy_smote = calculate_entropy(y_prob_smote)
    entropy_list_smote.append(entropy_smote)
    
    # Store metrics after SMOTE
    accuracy_list_smote.append(accuracy_score(y_test, y_pred_smote))
    precision_list_smote.append(precision_score(y_test, y_pred_smote))
    recall_list_smote.append(recall_score(y_test, y_pred_smote))
    f1_list_smote.append(f1_score(y_test, y_pred_smote))
    confusion_matrices_smote.append(confusion_matrix(y_test, y_pred_smote))
    
    # Apply ADASYN to the training data
    adasyn = ADASYN(random_state=42)
    X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)
    
    # Initialize and train the Decision Tree model with ADASYN
    dt_model_adasyn = DecisionTreeClassifier(random_state=42)
    dt_model_adasyn.fit(X_train_adasyn, y_train_adasyn)
    
    # Predictions and probabilities after ADASYN
    y_pred_adasyn = dt_model_adasyn.predict(X_test)
    y_prob_adasyn = dt_model_adasyn.predict_proba(X_test)
    
    # Calculate entropy after ADASYN
    entropy_adasyn = calculate_entropy(y_prob_adasyn)
    entropy_list_adasyn.append(entropy_adasyn)
    
    # Store metrics after ADASYN
    accuracy_list_adasyn.append(accuracy_score(y_test, y_pred_adasyn))
    precision_list_adasyn.append(precision_score(y_test, y_pred_adasyn))
    recall_list_adasyn.append(recall_score(y_test, y_pred_adasyn))
    f1_list_adasyn.append(f1_score(y_test, y_pred_adasyn))
    confusion_matrices_adasyn.append(confusion_matrix(y_test, y_pred_adasyn))
    
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Oversampling:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_smote)} After SMOTE:\n', confusion_matrices_smote[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_adasyn)} After ADASYN:\n', confusion_matrices_adasyn[-1])

# Calculate mean and standard deviation of each metric before oversampling
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after SMOTE
mean_accuracy_smote = np.mean(accuracy_list_smote)
std_accuracy_smote = np.std(accuracy_list_smote)
mean_precision_smote = np.mean(precision_list_smote)
std_precision_smote = np.std(precision_list_smote)
mean_recall_smote = np.mean(recall_list_smote)
std_recall_smote = np.std(recall_list_smote)
mean_f1_smote = np.mean(f1_list_smote)
std_f1_smote = np.std(f1_list_smote)
mean_entropy_smote = np.mean(entropy_list_smote)
std_entropy_smote = np.std(entropy_list_smote)

# Calculate mean and standard deviation of each metric after ADASYN
mean_accuracy_adasyn = np.mean(accuracy_list_adasyn)
std_accuracy_adasyn = np.std(accuracy_list_adasyn)
mean_precision_adasyn = np.mean(precision_list_adasyn)
std_precision_adasyn = np.std(precision_list_adasyn)
mean_recall_adasyn = np.mean(recall_list_adasyn)
std_recall_adasyn = np.std(recall_list_adasyn)
mean_f1_adasyn = np.mean(f1_list_adasyn)
std_f1_adasyn = np.std(f1_list_adasyn)
mean_entropy_adasyn = np.mean(entropy_list_adasyn)
std_entropy_adasyn = np.std(entropy_list_adasyn)

# Calculate mean confusion matrix before and after SMOTE/ADASYN
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_smote = np.mean(confusion_matrices_smote, axis=0)
mean_conf_matrix_adasyn = np.mean(confusion_matrices_adasyn, axis=0)

# Print results before oversampling
print('--- Before Oversampling ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after SMOTE
print('--- After SMOTE ---')
print('Mean Accuracy:', mean_accuracy_smote)
print('Accuracy Std Dev:', std_accuracy_smote)
print('Mean Precision:', mean_precision_smote)
print('Precision Std Dev:', std_precision_smote)
print('Mean Recall:', mean_recall_smote)
print('Recall Std Dev:', std_recall_smote)
print('Mean F1-score:', mean_f1_smote)
print('F1-score Std Dev:', std_f1_smote)
print('Mean Entropy:', mean_entropy_smote)
print('Entropy Std Dev:', std_entropy_smote)
print('Mean Confusion Matrix:\n', mean_conf_matrix_smote)

# Print results after ADASYN
print('--- After ADASYN ---')
print('Mean Accuracy:', mean_accuracy_adasyn)
print('Accuracy Std Dev:', std_accuracy_adasyn)
print('Mean Precision:', mean_precision_adasyn)
print('Precision Std Dev:', std_precision_adasyn)
print('Mean Recall:', mean_recall_adasyn)
print('Recall Std Dev:', std_recall_adasyn)
print('Mean F1-score:', mean_f1_adasyn)
print('F1-score Std Dev:', std_f1_adasyn)
print('Mean Entropy:', mean_entropy_adasyn)
print('Entropy Std Dev:', std_entropy_adasyn)
print('Mean Confusion Matrix:\n', mean_conf_matrix_adasyn)


Confusion Matrix for Fold 1 Before Oversampling:
 [[1152    7]
 [   7   13]]
Confusion Matrix for Fold 1 After SMOTE:
 [[1137   22]
 [   2   18]]
Confusion Matrix for Fold 1 After ADASYN:
 [[1137   22]
 [   2   18]]
Confusion Matrix for Fold 2 Before Oversampling:
 [[1153    6]
 [   7   13]]
Confusion Matrix for Fold 2 After SMOTE:
 [[1135   24]
 [   1   19]]
Confusion Matrix for Fold 2 After ADASYN:
 [[1136   23]
 [   2   18]]
Confusion Matrix for Fold 3 Before Oversampling:
 [[1149   10]
 [  10   10]]
Confusion Matrix for Fold 3 After SMOTE:
 [[1127   32]
 [   6   14]]
Confusion Matrix for Fold 3 After ADASYN:
 [[1127   32]
 [   6   14]]
Confusion Matrix for Fold 4 Before Oversampling:
 [[1152    7]
 [   9   11]]
Confusion Matrix for Fold 4 After SMOTE:
 [[1133   26]
 [   4   16]]
Confusion Matrix for Fold 4 After ADASYN:
 [[1135   24]
 [   4   16]]
Confusion Matrix for Fold 5 Before Oversampling:
 [[1149   10]
 [   9   11]]
Confusion Matrix for Fold 5 After SMOTE:
 [[1127   32]
 [  

In [12]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
import numpy as np

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
uncertainty_list = []  # List to store entropy for each fold
confusion_matrices = []

# Choose an oversampling method
# oversampler = SMOTE(random_state=42)
# oversampler = ADASYN(random_state=42)
oversampler = RandomOverSampler(random_state=42)

# Function to calculate entropy
def calculate_entropy(probabilities):
    entropy = -np.sum(probabilities * np.log(probabilities + 1e-9), axis=1)  # Add small value to avoid log(0)
    return np.mean(entropy)

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Apply oversampling
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
    
    # Initialize and train the Decision Tree model
    decision_tree_model = DecisionTreeClassifier(random_state=42)
    decision_tree_model.fit(X_train_resampled, y_train_resampled)
    
    # Predict probabilities and classes
    y_pred_tree = decision_tree_model.predict(X_test)
    y_prob_tree = decision_tree_model.predict_proba(X_test)
    
    # Calculate and store entropy
    entropy = calculate_entropy(y_prob_tree)
    uncertainty_list.append(entropy)
    
    # Store metrics
    accuracy_list.append(accuracy_score(y_test, y_pred_tree))
    precision_list.append(precision_score(y_test, y_pred_tree))
    recall_list.append(recall_score(y_test, y_pred_tree))
    f1_list.append(f1_score(y_test, y_pred_tree))
    
    # Store confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred_tree)
    confusion_matrices.append(conf_matrix)
    print(f'Confusion Matrix for Fold {len(confusion_matrices)}:\n', conf_matrix)

# Calculate mean and standard deviation of each metric
mean_accuracy = np.mean(accuracy_list)
std_accuracy = np.std(accuracy_list)
mean_precision = np.mean(precision_list)
std_precision = np.std(precision_list)
mean_recall = np.mean(recall_list)
std_recall = np.std(recall_list)
mean_f1 = np.mean(f1_list)
std_f1 = np.std(f1_list)

# Calculate mean and standard deviation of entropy
mean_entropy = np.mean(uncertainty_list)
std_entropy = np.std(uncertainty_list)

# Calculate mean confusion matrix
mean_conf_matrix = np.mean(confusion_matrices, axis=0)

# Print results
print('Mean Accuracy:', mean_accuracy)
print('Accuracy Std Dev:', std_accuracy)
print('Mean Precision:', mean_precision)
print('Precision Std Dev:', std_precision)
print('Mean Recall:', mean_recall)
print('Recall Std Dev:', std_recall)
print('Mean F1-score:', mean_f1)
print('F1-score Std Dev:', std_f1)
print('Mean Entropy:', mean_entropy)
print('Entropy Std Dev:', std_entropy)
print('Mean Confusion Matrix:\n', mean_conf_matrix)


Confusion Matrix for Fold 1:
 [[1152    7]
 [   8   12]]
Confusion Matrix for Fold 2:
 [[1148   11]
 [   7   13]]
Confusion Matrix for Fold 3:
 [[1150    9]
 [  11    9]]
Confusion Matrix for Fold 4:
 [[1154    5]
 [  10   10]]
Confusion Matrix for Fold 5:
 [[1152    7]
 [  10   10]]
Confusion Matrix for Fold 6:
 [[1152    7]
 [  10   10]]
Confusion Matrix for Fold 7:
 [[1155    4]
 [   5   15]]
Confusion Matrix for Fold 8:
 [[1151    8]
 [  14    6]]
Confusion Matrix for Fold 9:
 [[1154    5]
 [   9   11]]
Confusion Matrix for Fold 10:
 [[1153    5]
 [  10   10]]
Mean Accuracy: 0.9862584619638237
Accuracy Std Dev: 0.0028583619077449853
Mean Precision: 0.608859464838567
Precision Std Dev: 0.09783920182148552
Mean Recall: 0.53
Recall Std Dev: 0.11445523142259599
Mean F1-score: 0.5637199197570715
F1-score Std Dev: 0.10080508295349175
Mean Entropy: -1.0000000822403707e-09
Entropy Std Dev: 1.9618310790006263e-25
Mean Confusion Matrix:
 [[1152.1    6.8]
 [   9.4   10.6]]


In [14]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE, ADASYN
import numpy as np

# Define entropy function
def calculate_entropy(probabilities):
    epsilon = 1e-10  # Small constant to avoid log(0)
    return -np.mean(np.sum(probabilities * np.log(probabilities + epsilon), axis=1))

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list_before = []
precision_list_before = []
recall_list_before = []
f1_list_before = []
entropy_list_before = []
confusion_matrices_before = []

accuracy_list_smote = []
precision_list_smote = []
recall_list_smote = []
f1_list_smote = []
entropy_list_smote = []
confusion_matrices_smote = []

accuracy_list_adasyn = []
precision_list_adasyn = []
recall_list_adasyn = []
f1_list_adasyn = []
entropy_list_adasyn = []
confusion_matrices_adasyn = []

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Initialize and train the Random Forest model without SMOTE/ADASYN
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)
    
    # Predictions and probabilities before SMOTE/ADASYN
    y_pred_before = rf_model.predict(X_test)
    y_prob_before = rf_model.predict_proba(X_test)
    
    # Calculate entropy before SMOTE/ADASYN
    entropy_before = calculate_entropy(y_prob_before)
    entropy_list_before.append(entropy_before)
    
    # Store metrics before SMOTE/ADASYN
    accuracy_list_before.append(accuracy_score(y_test, y_pred_before))
    precision_list_before.append(precision_score(y_test, y_pred_before))
    recall_list_before.append(recall_score(y_test, y_pred_before))
    f1_list_before.append(f1_score(y_test, y_pred_before))
    confusion_matrices_before.append(confusion_matrix(y_test, y_pred_before))
    
    # Apply SMOTE to the training data
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    
    # Initialize and train the Random Forest model with SMOTE
    rf_model_smote = RandomForestClassifier(random_state=42)
    rf_model_smote.fit(X_train_smote, y_train_smote)
    
    # Predictions and probabilities after SMOTE
    y_pred_smote = rf_model_smote.predict(X_test)
    y_prob_smote = rf_model_smote.predict_proba(X_test)
    
    # Calculate entropy after SMOTE
    entropy_smote = calculate_entropy(y_prob_smote)
    entropy_list_smote.append(entropy_smote)
    
    # Store metrics after SMOTE
    accuracy_list_smote.append(accuracy_score(y_test, y_pred_smote))
    precision_list_smote.append(precision_score(y_test, y_pred_smote))
    recall_list_smote.append(recall_score(y_test, y_pred_smote))
    f1_list_smote.append(f1_score(y_test, y_pred_smote))
    confusion_matrices_smote.append(confusion_matrix(y_test, y_pred_smote))
    
    # Apply ADASYN to the training data
    adasyn = ADASYN(random_state=42)
    X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)
    
    # Initialize and train the Random Forest model with ADASYN
    rf_model_adasyn = RandomForestClassifier(random_state=42)
    rf_model_adasyn.fit(X_train_adasyn, y_train_adasyn)
    
    # Predictions and probabilities after ADASYN
    y_pred_adasyn = rf_model_adasyn.predict(X_test)
    y_prob_adasyn = rf_model_adasyn.predict_proba(X_test)
    
    # Calculate entropy after ADASYN
    entropy_adasyn = calculate_entropy(y_prob_adasyn)
    entropy_list_adasyn.append(entropy_adasyn)
    
    # Store metrics after ADASYN
    accuracy_list_adasyn.append(accuracy_score(y_test, y_pred_adasyn))
    precision_list_adasyn.append(precision_score(y_test, y_pred_adasyn))
    recall_list_adasyn.append(recall_score(y_test, y_pred_adasyn))
    f1_list_adasyn.append(f1_score(y_test, y_pred_adasyn))
    confusion_matrices_adasyn.append(confusion_matrix(y_test, y_pred_adasyn))
    
    print(f'Confusion Matrix for Fold {len(confusion_matrices_before)} Before Oversampling:\n', confusion_matrices_before[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_smote)} After SMOTE:\n', confusion_matrices_smote[-1])
    print(f'Confusion Matrix for Fold {len(confusion_matrices_adasyn)} After ADASYN:\n', confusion_matrices_adasyn[-1])

# Calculate mean and standard deviation of each metric before oversampling
mean_accuracy_before = np.mean(accuracy_list_before)
std_accuracy_before = np.std(accuracy_list_before)
mean_precision_before = np.mean(precision_list_before)
std_precision_before = np.std(precision_list_before)
mean_recall_before = np.mean(recall_list_before)
std_recall_before = np.std(recall_list_before)
mean_f1_before = np.mean(f1_list_before)
std_f1_before = np.std(f1_list_before)
mean_entropy_before = np.mean(entropy_list_before)
std_entropy_before = np.std(entropy_list_before)

# Calculate mean and standard deviation of each metric after SMOTE
mean_accuracy_smote = np.mean(accuracy_list_smote)
std_accuracy_smote = np.std(accuracy_list_smote)
mean_precision_smote = np.mean(precision_list_smote)
std_precision_smote = np.std(precision_list_smote)
mean_recall_smote = np.mean(recall_list_smote)
std_recall_smote = np.std(recall_list_smote)
mean_f1_smote = np.mean(f1_list_smote)
std_f1_smote = np.std(f1_list_smote)
mean_entropy_smote = np.mean(entropy_list_smote)
std_entropy_smote = np.std(entropy_list_smote)

# Calculate mean and standard deviation of each metric after ADASYN
mean_accuracy_adasyn = np.mean(accuracy_list_adasyn)
std_accuracy_adasyn = np.std(accuracy_list_adasyn)
mean_precision_adasyn = np.mean(precision_list_adasyn)
std_precision_adasyn = np.std(precision_list_adasyn)
mean_recall_adasyn = np.mean(recall_list_adasyn)
std_recall_adasyn = np.std(recall_list_adasyn)
mean_f1_adasyn = np.mean(f1_list_adasyn)
std_f1_adasyn = np.std(f1_list_adasyn)
mean_entropy_adasyn = np.mean(entropy_list_adasyn)
std_entropy_adasyn = np.std(entropy_list_adasyn)

# Calculate mean confusion matrix before and after SMOTE/ADASYN
mean_conf_matrix_before = np.mean(confusion_matrices_before, axis=0)
mean_conf_matrix_smote = np.mean(confusion_matrices_smote, axis=0)
mean_conf_matrix_adasyn = np.mean(confusion_matrices_adasyn, axis=0)

# Print results before oversampling
print('--- Before Oversampling ---')
print('Mean Accuracy:', mean_accuracy_before)
print('Accuracy Std Dev:', std_accuracy_before)
print('Mean Precision:', mean_precision_before)
print('Precision Std Dev:', std_precision_before)
print('Mean Recall:', mean_recall_before)
print('Recall Std Dev:', std_recall_before)
print('Mean F1-score:', mean_f1_before)
print('F1-score Std Dev:', std_f1_before)
print('Mean Entropy:', mean_entropy_before)
print('Entropy Std Dev:', std_entropy_before)
print('Mean Confusion Matrix:\n', mean_conf_matrix_before)

# Print results after SMOTE
print('--- After SMOTE ---')
print('Mean Accuracy:', mean_accuracy_smote)
print('Accuracy Std Dev:', std_accuracy_smote)
print('Mean Precision:', mean_precision_smote)
print('Precision Std Dev:', std_precision_smote)
print('Mean Recall:', mean_recall_smote)
print('Recall Std Dev:', std_recall_smote)
print('Mean F1-score:', mean_f1_smote)
print('F1-score Std Dev:', std_f1_smote)
print('Mean Entropy:', mean_entropy_smote)
print('Entropy Std Dev:', std_entropy_smote)
print('Mean Confusion Matrix:\n', mean_conf_matrix_smote)

# Print results after ADASYN
print('--- After ADASYN ---')
print('Mean Accuracy:', mean_accuracy_adasyn)
print('Accuracy Std Dev:', std_accuracy_adasyn)
print('Mean Precision:', mean_precision_adasyn)
print('Precision Std Dev:', std_precision_adasyn)
print('Mean Recall:', mean_recall_adasyn)
print('Recall Std Dev:', std_recall_adasyn)
print('Mean F1-score:', mean_f1_adasyn)
print('F1-score Std Dev:', std_f1_adasyn)
print('Mean Entropy:', mean_entropy_adasyn)
print('Entropy Std Dev:', std_entropy_adasyn)
print('Mean Confusion Matrix:\n', mean_conf_matrix_adasyn)


Confusion Matrix for Fold 1 Before Oversampling:
 [[1155    4]
 [   8   12]]
Confusion Matrix for Fold 1 After SMOTE:
 [[1138   21]
 [   1   19]]
Confusion Matrix for Fold 1 After ADASYN:
 [[1139   20]
 [   2   18]]
Confusion Matrix for Fold 2 Before Oversampling:
 [[1153    6]
 [   6   14]]
Confusion Matrix for Fold 2 After SMOTE:
 [[1136   23]
 [   1   19]]
Confusion Matrix for Fold 2 After ADASYN:
 [[1137   22]
 [   1   19]]
Confusion Matrix for Fold 3 Before Oversampling:
 [[1153    6]
 [  11    9]]
Confusion Matrix for Fold 3 After SMOTE:
 [[1131   28]
 [   6   14]]
Confusion Matrix for Fold 3 After ADASYN:
 [[1131   28]
 [   7   13]]
Confusion Matrix for Fold 4 Before Oversampling:
 [[1156    3]
 [   7   13]]
Confusion Matrix for Fold 4 After SMOTE:
 [[1138   21]
 [   4   16]]
Confusion Matrix for Fold 4 After ADASYN:
 [[1137   22]
 [   4   16]]
Confusion Matrix for Fold 5 Before Oversampling:
 [[1154    5]
 [  10   10]]
Confusion Matrix for Fold 5 After SMOTE:
 [[1127   32]
 [  

In [15]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
import numpy as np

# Number of folds
n_splits = 10

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store metrics for each fold
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []
uncertainty_list = []  # List to store entropy for each fold
confusion_matrices = []

# Choose an oversampling method
# oversampler = SMOTE(random_state=42)
# oversampler = ADASYN(random_state=42)
oversampler = RandomOverSampler(random_state=42)

# Function to calculate entropy
def calculate_entropy(probabilities):
    entropy = -np.sum(probabilities * np.log(probabilities + 1e-9), axis=1)  # Add small value to avoid log(0)
    return np.mean(entropy)

# Loop through the StratifiedKFold splits
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Apply oversampling
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
    
    # Initialize and train the Random Forest model
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train_resampled, y_train_resampled)
    
    # Predict probabilities and classes
    y_pred_rf = rf_model.predict(X_test)
    y_prob_rf = rf_model.predict_proba(X_test)
    
    # Calculate and store entropy
    entropy = calculate_entropy(y_prob_rf)
    uncertainty_list.append(entropy)
    
    # Store metrics
    accuracy_list.append(accuracy_score(y_test, y_pred_rf))
    precision_list.append(precision_score(y_test, y_pred_rf))
    recall_list.append(recall_score(y_test, y_pred_rf))
    f1_list.append(f1_score(y_test, y_pred_rf))
    
    # Store confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred_rf)
    confusion_matrices.append(conf_matrix)
    print(f'Confusion Matrix for Fold {len(confusion_matrices)}:\n', conf_matrix)

# Calculate mean and standard deviation of each metric
mean_accuracy = np.mean(accuracy_list)
std_accuracy = np.std(accuracy_list)
mean_precision = np.mean(precision_list)
std_precision = np.std(precision_list)
mean_recall = np.mean(recall_list)
std_recall = np.std(recall_list)
mean_f1 = np.mean(f1_list)
std_f1 = np.std(f1_list)

# Calculate mean and standard deviation of entropy
mean_entropy = np.mean(uncertainty_list)
std_entropy = np.std(uncertainty_list)

# Calculate mean confusion matrix
mean_conf_matrix = np.mean(confusion_matrices, axis=0)

# Print results
print('Mean Accuracy:', mean_accuracy)
print('Accuracy Std Dev:', std_accuracy)
print('Mean Precision:', mean_precision)
print('Precision Std Dev:', std_precision)
print('Mean Recall:', mean_recall)
print('Recall Std Dev:', std_recall)
print('Mean F1-score:', mean_f1)
print('F1-score Std Dev:', std_f1)
print('Mean Entropy:', mean_entropy)
print('Entropy Std Dev:', std_entropy)
print('Mean Confusion Matrix:\n', mean_conf_matrix)


Confusion Matrix for Fold 1:
 [[1154    5]
 [   8   12]]
Confusion Matrix for Fold 2:
 [[1149   10]
 [   5   15]]
Confusion Matrix for Fold 3:
 [[1152    7]
 [  10   10]]
Confusion Matrix for Fold 4:
 [[1154    5]
 [   8   12]]
Confusion Matrix for Fold 5:
 [[1154    5]
 [   9   11]]
Confusion Matrix for Fold 6:
 [[1153    6]
 [   5   15]]
Confusion Matrix for Fold 7:
 [[1156    3]
 [   6   14]]
Confusion Matrix for Fold 8:
 [[1152    7]
 [  13    7]]
Confusion Matrix for Fold 9:
 [[1154    5]
 [   9   11]]
Confusion Matrix for Fold 10:
 [[1151    7]
 [   9   11]]
Mean Accuracy: 0.9879547428038208
Accuracy Std Dev: 0.002482261329814338
Mean Precision: 0.662392623716153
Precision Std Dev: 0.08490862023090356
Mean Recall: 0.59
Recall Std Dev: 0.11575836902790225
Mean F1-score: 0.620590287486006
F1-score Std Dev: 0.09301387283329499
Mean Entropy: 0.025642090602386685
Entropy Std Dev: 0.0028767933844409425
Mean Confusion Matrix:
 [[1152.9    6. ]
 [   8.2   11.8]]
